In [2]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    run_on_colab = True
else:
    print('Not running on CoLab')
    run_on_colab = False                                                                                                                                              
if run_on_colab:
    from google.colab import drive
    drive.mount('/drive', force_remount=True)

Running on CoLab
Mounted at /drive


In [3]:
!pip install ipython-autotime
%load_ext autotime

time: 216 µs (started: 2021-06-05 17:21:18 +00:00)


In [4]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.externals import joblib
import matplotlib.pyplot as plt

time: 769 ms (started: 2021-06-05 17:21:18 +00:00)


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [5]:
window_length=5
threshold=0.15
rocs=1
clf = None
scaler = None

time: 2.34 ms (started: 2021-06-05 17:21:20 +00:00)


In [7]:
train_df = pd.read_csv("/drive/My Drive/disertation/train_df_{}_rocs_{}.csv".format(threshold,rocs))
train_df = train_df.set_index("open_time")
train_df.drop(columns=["close"],inplace=True)
train_df

,sma5_diff,sma8_diff,sma13_diff,sma21_diff,ema5_diff,ema8_diff,ema13_diff,ema21_diff,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,ROC_1,STOCHk_14_3_3,STOCHd_14_3_3,CCI_14_0.015,ATRr_14,WILLR_14,AROOND_14,AROONU_14,AROONOSC_14,dema10_diff,BBB_5_2.0,BBU_5_2.0_diff,BBL_5_2.0_diff,BBM_5_2.0_diff,RVI_14,output_class
open_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-08-17 04:33:00,-0.000882,-0.000402,-0.000094,0.000409,-0.000701,-0.000509,-0.000204,0.000135,2.517063,0.710935,1.806128,52.037221,-0.156269,57.813583,64.134419,-1.973131,3.477987,-65.332640,64.285714,78.571429,14.285714,-0.000948,0.670908,-0.004240,0.002475,-0.000882,26.024920,0.0
2017-08-17 04:34:00,0.004152,0.005384,0.006029,0.006682,0.003980,0.004791,0.005539,0.006182,4.527372,2.176995,2.350377,73.578620,0.671042,68.018018,68.150843,285.454223,5.420447,0.000000,57.142857,100.000000,42.857143,0.003748,1.013689,-0.000896,0.009199,0.004152,58.444576,1.0
2017-08-17 04:35:00,0.004072,0.005260,0.006286,0.007118,0.003232,0.004401,0.005491,0.006408,6.349951,3.199659,3.150292,75.152240,0.087278,78.222453,68.018018,212.079230,5.292232,0.000000,50.000000,100.000000,50.000000,0.002882,1.362926,-0.002715,0.010859,0.004072,75.155465,1.0
2017-08-17 04:36:00,0.002876,0.004269,0.005676,0.006687,0.002155,0.003423,0.004707,0.005825,7.705534,3.644193,4.061340,75.152240,0.000000,100.000000,82.080157,156.805529,4.888177,0.000000,42.857143,100.000000,57.142857,0.001626,1.444458,-0.004325,0.010078,0.002876,75.155465,1.0
2017-08-17 04:37:00,0.001681,0.003138,0.005066,0.006256,0.001437,0.002663,0.004034,0.005296,8.679787,3.694758,4.985030,75.152240,0.000000,100.000000,92.740818,120.704989,4.516799,0.000000,35.714286,100.000000,64.285714,0.000732,1.319255,-0.004904,0.008266,0.001681,75.155465,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-29 15:07:00,0.000561,0.000571,0.000239,0.000047,0.000337,0.000346,0.000218,-0.000055,-3.690980,0.308423,-3.999402,47.534596,0.014855,50.769231,35.601140,21.098279,6.758089,-40.205128,64.285714,7.142857,-57.142857,0.000531,0.207261,-0.000475,0.001596,0.000561,56.326805,1.0
2018-07-29 15:08:00,0.000527,0.000675,0.000470,0.000239,0.000352,0.000418,0.000351,0.000124,-3.128508,0.696716,-3.825224,49.653041,0.019113,60.592056,48.294218,59.137727,6.430368,-31.557165,57.142857,0.000000,-57.142857,0.000517,0.223438,-0.000590,0.001644,0.000527,61.932170,1.0
2018-07-29 15:09:00,0.000366,0.000728,0.000598,0.000375,0.000337,0.000444,0.000432,0.000252,-2.551659,1.018851,-3.570511,51.350797,0.015336,67.862907,59.741398,65.158862,6.421771,-24.648986,50.000000,0.000000,-50.000000,0.000465,0.129509,-0.000281,0.001013,0.000366,66.525812,1.0


time: 4.51 s (started: 2021-06-05 17:21:42 +00:00)


In [8]:
train_df.index = pd.to_datetime(train_df.index)

time: 118 ms (started: 2021-06-05 17:21:48 +00:00)


In [9]:
data = train_df.to_numpy()
X = data[:,:-1]
Y = data[:,-1]

X.shape, Y.shape

((498879, 27), (498879,))

time: 10.5 ms (started: 2021-06-05 17:21:48 +00:00)


In [10]:
def extend_dataset_with_window_length(X, Y, window_length=5):
    new_x = []
    for i in range(len(X) - window_length+1):
        lst = []
        for j in range(i, i+window_length):
            lst.extend(X[j])
        new_x.append(lst)
    return np.array(new_x), Y[window_length-1:]

time: 4.89 ms (started: 2021-06-05 17:21:50 +00:00)


In [11]:
X, Y = extend_dataset_with_window_length(X,Y)
X.shape, Y.shape

((498875, 135), (498875,))

time: 31 s (started: 2021-06-05 17:21:51 +00:00)


In [12]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

time: 696 ms (started: 2021-06-05 17:22:22 +00:00)


In [13]:
scaler_path = "/drive/My Drive/disertation/scaler_logistic.save"
if scaler:
    joblib.dump(scaler, scaler_path)
else:
    scaler = joblib.load(scaler_path)

time: 12.9 ms (started: 2021-06-05 17:22:23 +00:00)


In [14]:
def get_subsample_same_distribution(X,Y, p=0.1):
    ys = pd.Series(Y)
    vs = ys.value_counts()
    original_indexes = []
    for idx in vs.index:
        required = int(p * vs[idx])
        print(required)
        original_indexes.extend(ys[ys==idx].sample(n=required, replace=False).index)
    return X[original_indexes], Y[original_indexes]

time: 8.19 ms (started: 2021-06-05 17:22:23 +00:00)


In [15]:
a,b = np.unique(Y, return_counts=True)
print("buy hold sell %")
print(b)
print(b / sum(b))

buy hold sell %
[ 60627 377448  60800]
[0.12152744 0.75659835 0.12187422]
time: 27.1 ms (started: 2021-06-05 17:22:23 +00:00)


In [16]:
def get_equal_class_distribution(X,Y):
    ys = pd.Series(Y)
    vs = ys.value_counts()
    required_of_each_class = vs.min()
    original_indexes = []
    for idx in vs.index:
        original_indexes.extend(ys[ys==idx].sample(n=required_of_each_class, replace=False).index)
    return X[original_indexes], Y[original_indexes]

time: 6.71 ms (started: 2021-06-05 17:22:23 +00:00)


In [17]:
X,Y = get_equal_class_distribution(X,Y)
#X, Y = get_subsample_same_distribution(X,Y)
a,b = np.unique(Y, return_counts=True)
print("buy hold sell %")
print(b)
print(b / sum(b))

buy hold sell %
[60627 60627 60627]
[0.33333333 0.33333333 0.33333333]
time: 188 ms (started: 2021-06-05 17:22:31 +00:00)


In [18]:
clf = LogisticRegression(n_jobs=-1, verbose=True, max_iter=10000)
clf.fit(X, Y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  8.2min finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=True,
                   warm_start=False)

time: 8min 14s (started: 2021-06-05 17:22:47 +00:00)


In [22]:
clf.n_iter_

array([2076], dtype=int32)

time: 8.32 ms (started: 2021-06-05 17:32:00 +00:00)


In [20]:
print(clf.score(X, Y))
Y_train_predicted = clf.predict(X)

print(np.array(np.unique(Y, return_counts=True)).T)

a,b = np.unique(Y, return_counts=True)
print("buy hold sell %")
print(b / sum(b))

print(np.array(np.unique(Y_train_predicted, return_counts=True)).T)

a,b = np.unique(Y_train_predicted, return_counts=True)
print("buy hold sell %")
print(b / sum(b))

0.5254534558310104
[[0.0000e+00 6.0627e+04]
 [1.0000e+00 6.0627e+04]
 [2.0000e+00 6.0627e+04]]
buy hold sell %
[0.33333333 0.33333333 0.33333333]
[[0.0000e+00 5.2825e+04]
 [1.0000e+00 7.8779e+04]
 [2.0000e+00 5.0277e+04]]
buy hold sell %
[0.29043715 0.43313485 0.27642799]
time: 256 ms (started: 2021-06-05 17:31:01 +00:00)


In [21]:
Y_train_predicted = clf.predict(X)
print(metrics.confusion_matrix(Y, Y_train_predicted))

# Print the precision and recall, among other metrics
print(metrics.classification_report(Y, Y_train_predicted, digits=3))

[[27414 18228 14985]
 [ 9517 41987  9123]
 [15894 18564 26169]]
              precision    recall  f1-score   support

         0.0      0.519     0.452     0.483     60627
         1.0      0.533     0.693     0.602     60627
         2.0      0.520     0.432     0.472     60627

    accuracy                          0.525    181881
   macro avg      0.524     0.525     0.519    181881
weighted avg      0.524     0.525     0.519    181881

time: 708 ms (started: 2021-06-05 17:31:01 +00:00)


In [23]:
clf_file = "/drive/My Drive/disertation/logistic_regression_attempt_1_dataset_balanced.pickle"
if clf:
    pickle.dump(clf, open(clf_file, 'wb'))
else:
    clf = pickle.load(open(clf_file, "rb"))

time: 10.7 ms (started: 2021-06-05 17:33:51 +00:00)


In [27]:
tune_df = pd.read_csv("/drive/My Drive/disertation/tune_df_{}_rocs_{}.csv".format(threshold,rocs))
tune_df = tune_df.set_index("open_time")

tune_df.drop(columns="close", inplace=True)

data = tune_df.to_numpy()
X_tune = data[:,:-1]
Y_tune = data[:,-1]

X_tune, Y_tune = extend_dataset_with_window_length(X_tune, Y_tune)

time: 59.8 s (started: 2021-06-05 17:35:47 +00:00)


In [28]:
X_tune = scaler.transform(X_tune)

time: 651 ms (started: 2021-06-05 17:36:53 +00:00)


In [29]:
a,b = np.unique(Y_tune, return_counts=True)
print("buy hold sell %")
print(b / sum(b))
Y_tune_predicted = clf.predict(X_tune)
print(metrics.confusion_matrix(Y_tune, Y_tune_predicted))

print(metrics.classification_report(Y_tune, Y_tune_predicted, digits=3))


buy hold sell %
[0.03926218 0.9223085  0.03842931]
[[  9083  21213   8878]
 [ 26999 862216  31022]
 [  8193  21063   9087]]
              precision    recall  f1-score   support

         0.0      0.205     0.232     0.218     39174
         1.0      0.953     0.937     0.945    920237
         2.0      0.185     0.237     0.208     38343

    accuracy                          0.882    997754
   macro avg      0.448     0.469     0.457    997754
weighted avg      0.894     0.882     0.888    997754

time: 3.59 s (started: 2021-06-05 17:36:57 +00:00)


In [32]:
# clear ram
del tune_df
del X_tune
del Y_tune

NameError: ignored

time: 20.7 ms (started: 2021-06-05 17:38:42 +00:00)


In [33]:
test_df = pd.read_csv("/drive/My Drive/disertation/test_df_{}_rocs_{}.csv".format(threshold,rocs))
test_df = test_df.set_index("open_time")

test_df.drop(columns="close", inplace=True)

data = test_df.to_numpy()
X_test = data[:,:-1]
Y_test = data[:,-1]

X_test, Y_test = extend_dataset_with_window_length(X_test, Y_test)

time: 25.3 s (started: 2021-06-05 17:38:54 +00:00)


In [34]:
X_test = scaler.transform(X_test)

time: 346 ms (started: 2021-06-05 17:39:26 +00:00)


In [35]:
a,b = np.unique(Y_test, return_counts=True)
print("buy hold sell %")
print(b / sum(b))
Y_test_predicted = clf.predict(X_test)
print(metrics.confusion_matrix(Y_test, Y_test_predicted))

print(metrics.classification_report(Y_test, Y_test_predicted, digits=3))

buy hold sell %
[0.06405412 0.87312052 0.06282536]
[[ 13684   5619  12652]
 [ 77961 290281  67336]
 [ 14300   5332  11710]]
              precision    recall  f1-score   support

         0.0      0.129     0.428     0.198     31955
         1.0      0.964     0.666     0.788    435578
         2.0      0.128     0.374     0.190     31342

    accuracy                          0.633    498875
   macro avg      0.407     0.489     0.392    498875
weighted avg      0.858     0.633     0.713    498875

time: 1.9 s (started: 2021-06-05 17:39:26 +00:00)
